# 7.1.5 幅值估计和填零

此示例展示了如何通过补零方法提高对正弦信号振幅估计的精度。在离散傅里叶变换（DFT）中，频率分辨率由采样率与输入时间序列的长度 N的比值决定，即$\frac{F_s}{N}$。当尝试估计某正弦波的振幅时，若其频率不恰好对应于DFT的一个频率 bin，则振幅估计可能会出现偏差。通过在执行DFT前对数据进行零填充，通常能够增强振幅估计的精确度。

本例中，我们将创建一个由两个正弦波组成的信号，其频率分别为100 Hz 和202.5 Hz。设定采样率为1000Hz，信号由1000个采样点构成。通过这个过程，我们将演示如何通过零填充提高振幅估计的准确性。

获取信号的 DFT。DFT bin 的间距为 1 Hz。相应地，100 Hz 正弦波对应到一个 DFT bin，但 202.5 Hz 正弦波无法对应。
由于信号是实数值信号，此处只使用 DFT 的正频率来估计振幅。按输入信号的长度缩放 DFT，并将 0 和奈奎斯特之外的所有频率乘以 2。
绘制结果并与已知振幅进行比较。

In [ ]:
using MAT
# using Plots
using TyPlot
using TySignalProcessing
using TyMath
using TyBase
using TyStatistics

Fs = 1000;
t = 0:1/Fs:1-1/Fs;
x = cos.(2 * π * 100 * t) + sin.(2 * π * 202.5 * t);

xdft = fft(x);
xdft = xdft[1:length(x)÷2+1];
# 归一化
xdft ./= length(x);

# 除了第一个和最后一个元素，其他元素乘以2（由于FFT的对称性）
xdft[2:end-1] .*= 2;

# 创建频率向量
freq = 0:Fs/length(x):Fs/2;
figure(1)
plot(freq, abs.(xdft), label="Magnitude")
hold("on")
plot(freq, ones(length(x)÷2+1), label="Reference", linewidth=2)

# 添加标签
xlabel("Frequency (Hz)")
ylabel("Amplitude")


PyObject Text(55.72916666666667, 0.5, 'Amplitude')

在100 Hz的正弦波的情况下，振幅估计是精确的，因为它的频率恰好对应于DFT的一个频率bin。然而，对于202.5 Hz的正弦波，情况就不同了，因为它的频率与DFT的bin并不对应，导致振幅估计的不准确。
为了提高对202.5 Hz正弦波振幅的估计精度，我们可以采用填零的方法对DFT进行插值。通过在原始数据后面添加零值，我们可以获得更精细的频率分辨率，从而对信号的振幅进行更准确的估计。需要注意的是，填零操作并不会改变DFT的频谱分辨率，后者依然由采样数和采样率决定。

接下来，我们将DFT的长度增加到2000，也就是原始信号长度的两倍。这样，DFT bin的间距变为 Fs/2000=0.5Hz。在这个新的分辨率下，202.5 Hz的正弦波的能量将恰好落在一个DFT bin中。我们将执行DFT，并绘制出振幅估计图。通过填零，我们将采样点的数量扩展到2000个，以实现更精确的振幅测量。

In [ ]:
lpad = 2 * length(x);

# 创建零填充的信号
x_padded = [x; zeros(lpad - length(x))];

# 进行FFT
xdft = fft(x_padded);

# 保留一半的频谱（由于对称性）
xdft = xdft[1:div(lpad, 2) + 1];

# 归一化
xdft ./= length(x);

# 除了第一个和最后一个元素，其他元素乘以2（由于FFT的对称性）
xdft[2:end-1] .*= 2;

# 创建频率向量
freq = 0:Fs/lpad:Fs/2;

figure(2)
# 绘制频谱图
plot(freq, abs.(xdft), label="Magnitude")
hold("on")
# 绘制参考线
plot(freq,ones(div(lpad, 2) + 1), label="Reference", linewidth=2)

# 添加标签
xlabel("Frequency (Hz)")
ylabel("Amplitude")

PyObject Text(55.72916666666667, 0.5, 'Amplitude')